In [1]:
from pm4py.algo.simulation.tree_generator import algorithm  as tree_gen

from pm4py.algo.simulation.playout.process_tree import algorithm as tree_playout

from pm4py.objects.process_tree.exporter import exporter as ptml_exporter

In [2]:
min_variants = 80
max_variants = 160
min_length = 5

In [3]:
def get_percentages_tree(tree):
    seq = str(tree).count('->')
    par = str(tree).count('+')
    cho = str(tree).count('X')
    return ([(seq/(seq+par+cho)), (cho/(seq+par+cho)), (par/(seq+par+cho))])

In [4]:
def distance(list1, list2):
    """Distance between two vectors."""
    squares = [(p-q) ** 2 for p, q in zip(list1, list2)]
    return sum(squares) ** .5

In [5]:
def pick_most_optimal(percentages, number_of_variants, minimum_lengths, opt_percentages, min_variants, max_variants, min_length): #returns index of the best fitting tree
    distances = []
    original_indices = []
    for i in range(0, len(percentages)):
        if min_variants <= number_of_variants[i] <= max_variants and minimum_lengths[i] > min_length:
            original_indices.append(i)
            distances.append(distance(percentages[i],opt_percentages))
    if len(distances) == 0:
        print("ERROR", opt_percentages)
        return False, 0
    else:
        min_value = min(distances)
        min_index = distances.index(min_value)
        return True, original_indices[min_index]
        

In [10]:
parallel_options = [0.0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50]

#changed it so it guarantees to only look at trees with exact percentages

for parallel in parallel_options:
    trees = []
    percentages = [] #the percentages of each of the operators in a tree 
    number_of_variants = []
    minimum_lengths = []
    filename = "Trees/tree_par" + "{:.2f}".format(parallel)[2:] + ".ptml"
    print(filename)
    choice = 0.5 - parallel
    opt_percentages = [0.5, choice, parallel]
    for mode_act in range(10,25):
        min_act = mode_act - 1
        max_act = mode_act + 1
        for i in range(0,150):    
            parameters = {"sequence":0.5, "choice":choice, "parallel":parallel, "loop":0.0,
                          "or":0.0,"mode":mode_act, "min":min_act, "max":max_act,"silent":0.0, "duplicate":0.0, 
                          "lt_dependency": 0.0, "infrequent":0.0,"no_models":1, "unfold":0, "max_repeat":3}
            tree = tree_gen.apply(parameters=parameters)
            if round(get_percentages_tree(tree)[2],2) == opt_percentages[2]:
                trees.append(tree)
                percentages.append(get_percentages_tree(tree))
                playout_variant = tree_playout.Variants.EXTENSIVE
                param = tree_playout.Variants.EXTENSIVE.value.Parameters
                variants = tree_playout.apply(tree, variant=playout_variant, parameters={param.MAX_TRACE_LENGTH: 200, param.MAX_LOOP_OCC: 3, param.MAX_LIMIT_NUM_TRACES: 200})
                number_of_variants.append(len(variants))
                minimum_lengths.append(min(map(len, variants)))
    success, index = pick_most_optimal(percentages, number_of_variants, minimum_lengths, opt_percentages, min_variants, max_variants, min_length)
    if success == False:
        print("PROBLEM")
    else:
        print("Saved")
        ptml_exporter.apply(trees[index], filename)
    

Trees/tree_par45.ptml
Saved


In [6]:
'''parallel_options = [0.45]



for parallel in parallel_options:
    trees = []
    percentages = [] #the percentages of each of the operators in a tree 
    number_of_variants = []
    minimum_lengths = []
    filename = "Trees/tree_par" + "{:.2f}".format(parallel)[2:] + ".ptml"
    print(filename)
    choice = 0.5 - parallel
    opt_percentages = [0.5, choice, parallel]
    for mode_act in range(10,25):
        min_act = mode_act - 1
        max_act = mode_act + 1
        for i in range(0,300):    
            parameters = {"sequence":0.5, "choice":choice, "parallel":parallel, "loop":0.0,
                          "or":0.0,"mode":mode_act, "min":min_act, "max":max_act,"silent":0.0, "duplicate":0.0, 
                          "lt_dependency": 0.0, "infrequent":0.0,"no_models":1, "unfold":0, "max_repeat":3}
            tree = tree_gen.apply(parameters=parameters)
            trees.append(tree)
            percentages.append(get_percentages_tree(tree))
            playout_variant = tree_playout.Variants.EXTENSIVE
            param = tree_playout.Variants.EXTENSIVE.value.Parameters
            variants = tree_playout.apply(tree, variant=playout_variant, parameters={param.MAX_TRACE_LENGTH: 200, param.MAX_LOOP_OCC: 3, param.MAX_LIMIT_NUM_TRACES: 200})
            number_of_variants.append(len(variants))
            minimum_lengths.append(min(map(len, variants)))
    success, index = pick_most_optimal(percentages, number_of_variants, minimum_lengths, opt_percentages, min_variants, max_variants, min_length)
    if success == False:
        print("PROBLEM")
    else:
        print("Saved")
        ptml_exporter.apply(trees[index], filename)
    
'''    

Trees/tree_par20.ptml
Saved
Trees/tree_par30.ptml
Saved
Trees/tree_par35.ptml


KeyboardInterrupt: 